In [ ]:
# please make sure that your GPU RAM is lager than 25GB 
PATH = 'VRDL/VRDL_hw2' #you should key in the path of folder that contain this ipynb file
'''
for example, in my setting, I put this ipynb file folder in 'VRDL/VRDL_HW2' on My Drive
then PATH will be
'VRDL/VRDL_HW2'
'''

from google.colab import drive
drive._mount('/content/drive/')

!ls gdrive/MyDrive/
!nvidia-smi

import os
os.chdir('./drive/MyDrive/')

os.chdir(f'./{PATH}')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
ls: cannot access 'gdrive/MyDrive/': No such file or directory
Sun Nov 21 06:54:36 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                     

In [ ]:
import torch
print(torch.__version__)
import numpy
numpy.version.version

1.10.0


'1.19.5'

In [ ]:
# install some requirement
# please note that after reinstall the version of pytorch
# the kernel will restart because you update the version of pytorch.
# then you don't need to rerun this cell again!
!pip install -q condacolab
import condacolab
condacolab.install()
!conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:24
🔁 Restarting kernel...
Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | WARNING conda.core.solve:_add_specs(611): pinned spec cudatoolkit=11.1 conflicts with explicit specs.  Overriding pinned spec.
/ - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cudatoolkit=10.2
    - pytorch
    - torchaudio
    - torchvision


The following packages will be downloaded:

    package                    |          

In [ ]:
%cd dataset
!unzip -q train.zip
!unzip -q test.zip
!mv train train
!mv test test
%cd ..

/content/drive/MyDrive/VRDL/VRDL_hw2/dataset
replace test/100000.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
mv: cannot move 'test' to a subdirectory of itself, 'test/test'
/content/drive/My Drive/VRDL/VRDL_hw2


In [ ]:
#dataloader
import random as rd
import torch
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader
import py_files.utils as utils
from py_files.numloader import NumberDataset


rd.seed(0)
train_set = NumberDataset(is_train=True)
print(len(train_set))
ratio = 0.9
train_len = int(len(train_set)*ratio)
split_num = [train_len, len(train_set) - train_len]
train_set, val_set = data.random_split(train_set, split_num)
test_set = NumberDataset(is_train=False)




Testing set 13068


In [ ]:
BATCH_SIZE = 2
train_loader = DataLoader(train_set, 
                          batch_size = BATCH_SIZE,
                          shuffle = True,
                          pin_memory = True,
                          num_workers = 0,
                          collate_fn=utils.collate_fn)

val_loader = DataLoader(val_set, 
                          batch_size = BATCH_SIZE,
                          shuffle = False,
                          pin_memory = True,
                          num_workers = 0,
                          collate_fn=utils.collate_fn)

test_loader = DataLoader(test_set, 
                          batch_size = 1,
                          shuffle = False,
                          pin_memory = True,
                          collate_fn=utils.collate_fn)



In [ ]:
# https://www.uj5u.com/qita/27937.html
import torch
import torch.nn as nn
import torchvision
import torchvision.models as models
import py_files
from py_files import utils
from py_files import engine
from py_files.engine import train_one_epoch, evaluate
from tqdm import tqdm
import time
import json
from torchvision.models.detection.rpn import AnchorGenerator

## 1. Retinanet with backbone resnet152

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
backbone = nn.Sequential(*list(models.resnet152(pretrained=True).children())[:-2]).to(device)
backbone.out_channels = 2048
anchor_generator = AnchorGenerator(
            sizes=((32, 64, 128, 256, 512),),
            aspect_ratios=((0.5, 1.0, 2.0),)
)
model = models.detection.RetinaNet(backbone,
                                    num_classes=10,
                                    anchor_generator=anchor_generator
                                    )

model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]

# SGD
optimizer = torch.optim.SGD(params, 
                lr=0.0005,
                momentum=0.9, 
                weight_decay=5e-4)

# cos warmup annealing
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=1, T_mult=2)

epochs = 31

for epoch in range(0, epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=1000)
    # save model parameter
    torch.save(model, f'./model_para/retinanet{epoch}.pt')
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset    
    evaluate(model, val_loader, device=device)  
    print("="*60)  
print('finally finished.......QQQQQQ')


/usr/local/lib/python3.7/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272126608/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Epoch: [0]  [    0/15031]  eta: 5:30:01  lr: 0.000001  loss: 0.4030 (0.4030)  classification: 0.1317 (0.1317)  bbox_regression: 0.2712 (0.2712)  time: 1.3173  data: 0.0299  max mem: 8727
Epoch: [0]  [ 1000/15031]  eta: 2:55:20  lr: 0.000500  loss: 0.3683 (0.4014)  classification: 0.1108 (0.1433)  bbox_regression: 0.2619 (0.2582)  time: 0.7694  data: 0.0045  max mem: 12610
Epoch: [0]  [ 2000/15031]  eta: 2:42:30  lr: 0.000500  loss: 0.4056 (0.3925)  classification: 0.1343 (0.1370)  bbox_regression: 0.2330 (0.2555)  time: 0.7356  data: 0.0041  max mem: 12611
Epoch: [0]  [ 3000/15031]  eta: 2:30:07  lr: 0.000500  loss: 0.3313 (0.3944)  classification: 0.0947 (0.1377)  bbox_regression: 0.2355 (0.2567)  time: 0.7506  data: 0.0042  max mem: 12612
Epoch: [0]  [ 4000/15031]  eta: 2:17:27  lr: 0.000500  loss: 0.3767 (0.3969)  classification: 0.1528 (0.1399)  bbox_regression: 0.2496 (0.2569)  time: 0.7434  data: 0.0045  max mem: 12612
Epoch: [0]  [ 5000/15031]  eta: 2:04:51  lr: 0.000500  loss: 

## 2. fasterRCNN with backbone resnet152

In [ ]:
backbone=nn.Sequential(*list(models.resnet152(pretrained=True).children())[:-2]).to(device)
backbone.out_channels = 2048
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                    aspect_ratios=((0.5, 1.0, 2.0),))
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                            output_size=7,
                            sampling_ratio=2)

model = models.detection.FasterRCNN(backbone,
            num_classes=10,
            rpn_anchor_generator=anchor_generator,
            box_roi_pool=roi_pooler
            )

model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]

# SGD
optimizer = torch.optim.SGD(params, 
                lr=0.0005,
                momentum=0.9, 
                weight_decay=5e-4)

# cos warmup annealing
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=1, T_mult=2)

epochs = 31

for epoch in range(0, epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=1000)
    # save model parameter
    torch.save(model, f'./model_para/retinanet{epoch}.pt')
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset    
    evaluate(model, val_loader, device=device)  
    print("="*60)  
print('finally finished.......QQQQQQ')


## 3. try to ensemble the output of different models

In [ ]:
model1 = models.detection.retinanet_resnet50_fpn(pretrained=False,
                                                 progress=True, 
                                                 num_classes=10,
                                                 pretrained_backbone=True)
model2 = models.detection.retinanet_resnet50_fpn(pretrained=False,
                                                 progress=True, 
                                                 num_classes=10,
                                                 pretrained_backbone=True)
model1 = torch.load('./model_para/retinanet3.pt')
model2 = torch.load('./model_para/retinanet5-2.pt')

model1.to(device)
model2.to(device)

final_record = []

start_time = time.time()
for img, img_id in tqdm(test_loader):
    prediction=[]
    model1.eval()
    model2.eval()
    with torch.no_grad():
        prediction1 = model1([img[0].to(device)])
        prediction2 = model2([img[0].to(device)])
        prediction = prediction1

        prediction[0]['boxes'] = torch.cat((prediction[0]['boxes'], prediction2[0]['boxes']), 0)
        prediction[0]['image_id'] = torch.cat((prediction[0]['image_id'], prediction2[0]['image_id']), 0)
        prediction[0]['labels'] = torch.cat((prediction[0]['labels'], prediction2[0]['labels']), 0)
        prediction[0]['scores'] = torch.cat((prediction[0]['scores'], prediction2[0]['scores']), 0)
        

    final_indice = torchvision.ops.batched_nms(boxes=prediction[0]['boxes'], 
                                               scores=prediction[0]['scores'], 
                                               idxs=prediction[0]['labels'], 
                                               iou_threshold=0.5) 
    
    for i in final_indice:
        i = i.item()
        box_info = {}
        # image_id
        box_info['image_id'] = img_id[0]
        # score
        box_info['score'] = prediction[0]['scores'][i].item()
        # category_id
        label = prediction[0]['labels'][i].item()
        box_info['category_id'] = label
        # bbox
        x1 = prediction[0]['boxes'][i][0].item()
        y1 = prediction[0]['boxes'][i][1].item()
        x2 = prediction[0]['boxes'][i][2].item()
        y2 = prediction[0]['boxes'][i][3].item()
        width = x2 - x1
        height = y2 - y1
        box_info['bbox'] = [x1, y1, width, height]
        final_record.append(box_info)

end_time  = time.time()
print("\nInference time per image: ", (end_time - start_time) / len(test_loader))

json_obj = json.dumps(final_record, indent=4)
with open('answer.json', 'w') as f:
    f.write(json_obj)

## evaluate

In [ ]:
#evaluate

model = models.detection.retinanet_resnet50_fpn(pretrained=False,
                                                progress=True, 
                                                num_classes=10,
                                                pretrained_backbone=True)

backbone=nn.Sequential(*list(models.resnet152(pretrained=True).children())[:-2]).to(device)
backbone.out_channels = 2048
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                    aspect_ratios=((0.5, 1.0, 2.0),))
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                            output_size=7,
                            sampling_ratio=2)

model = models.detection.FasterRCNN(backbone,
            num_classes=10,
            rpn_anchor_generator=anchor_generator,
            box_roi_pool=roi_pooler
            )

model.to(device)
# model = models.detection.RetinaNet(backbone,
#                                     num_classes=10,
#                                     anchor_generator=anchor_generator
#                                     )
# model = torch.load('./model_para/152_retinanet5.pt')
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

final_record = []

start_time = time.time()
for img, img_id in tqdm(test_loader):
    model.eval()
    with torch.no_grad():
        prediction = model([img[0].to(device)])

    for i in range(prediction[0]['boxes'].cpu().shape[0]):
        box_info = {}
        # image_id
        box_info['image_id'] = img_id[0]
        # score
        box_info['score'] = prediction[0]['scores'][i].item()
        # category_id
        label = prediction[0]['labels'][i].item()
        box_info['category_id'] = label
        # bbox
        x1 = prediction[0]['boxes'][i][0].item()
        y1 = prediction[0]['boxes'][i][1].item()
        x2 = prediction[0]['boxes'][i][2].item()
        y2 = prediction[0]['boxes'][i][3].item()
        width = x2 - x1
        height = y2 - y1
        box_info['bbox'] = [x1, y1, width, height]
        final_record.append(box_info)

end_time  = time.time()
print("\nInference time per image: ", (end_time - start_time) / len(test_loader))

json_obj = json.dumps(final_record, indent=4)
with open('answer.json', 'w') as f:
    f.write(json_obj)

100%|██████████| 13068/13068 [29:49<00:00,  7.30it/s]


Inference time per image:  17.893013916015626
